In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import JsonOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS, Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_community.vectorstores.faiss import DistanceStrategy
from time import perf_counter, process_time
from langchain.prompts import HumanMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import AzureChatOpenAI
from langchain.chains import LLMChain
from langchain.document_loaders import TextLoader, JSONLoader, PyPDFLoader, AzureBlobStorageContainerLoader, AzureAIDataLoader
from langchain.text_splitter import CharacterTextSplitter, SpacyTextSplitter
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts.prompt import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import ResponseSchema, StructuredOutputParser, CommaSeparatedListOutputParser, JsonOutputKeyToolsParser
from langchain.memory import VectorStoreRetrieverMemory
import os

In [5]:
embeddings = FastEmbedEmbeddings()

c:\Users\alvar\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
README.md: 100%|██████████| 28.0/28.0 [00:00<?, ?B/s]s]
tokenizer_config.json: 100%|██████████| 1.24k/1.24k [00:00<00:00, 1.24MB/s]
c:\Users\alvar\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alvar\AppData\Local\Temp\fastembed_cache\models--qdrant--bge-small-en-v1.5-onnx-q. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingf

In [6]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=2000,
    chunk_overlap=0
)

In [7]:
file_path = "./data/recetas.txt"

#Carga documento
loader = TextLoader(file_path)

#Aplica el text_splitter al documento
docs = loader.load_and_split(
    text_splitter=text_splitter
)
print(f"Se han generado {len(docs)} documentos")

Se han generado 6 documentos


In [8]:
index_path = "./index/index_recetas"

if os.path.exists(index_path):
    vectorstore = Chroma(
    persist_directory=index_path,
    embedding_function=embeddings
)
    print(f"Vectorstore cargado de {index_path}")
else:
    
    vectorstore = Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        persist_directory=index_path
        )
    print(f" Se ha creado un nuevo vectorstore {index_path}")

 Se ha creado un nuevo vectorstore ./index/index_recetas


In [9]:
retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": 4
        }
)

In [10]:
print(retriever.invoke("test")[0].page_content, flush=True)

Bienvenido a este libro de recetas de cocina, un compendio culinario que busca deleitar tus sentidos y despertar tu creatividad en la cocina. La comida es mucho mÃ¡s que una simple necesidad fisiolÃ³gica; es un arte, una forma de expresiÃ³n y, sobre todo, una experiencia sensorial que nos conecta con nuestra cultura, tradiciones y con quienes nos rodean.
En estas pÃ¡ginas, encontrarÃ¡s una amplia variedad de recetas cuidadosamente seleccionadas para satisfacer todos los gustos y necesidades, desde platos reconfortantes y tradicionales hasta creaciones innovadoras que desafÃ­an los lÃ­mites de la imaginaciÃ³n culinaria. Desde deliciosos entrantes hasta postres exquisitos, cada receta ha sido elaborada con pasiÃ³n y atenciÃ³n al detalle, con el objetivo de inspirarte a explorar en la cocina y descubrir nuevas formas de disfrutar de la comida.
La cocina es un espacio de experimentaciÃ³n y aprendizaje constante, donde incluso los errores pueden convertirse en oportunidades para descubrir n